Test on the Graph explorer:
* https://developers.facebook.com/tools/explorer
* `ads_archive?fields=ad_creative_body%2Cad_creation_time%2Cad_creative_link_caption%2Cad_creative_link_description%2Ccurrency%2Cfunding_entity%2Cimpressions%2Cad_snapshot_url%2Cpage_id%2Cpage_name%2Cspend&search_terms=''&ad_reached_countries=['FR']&limit=25`

Documentation:
* https://www.facebook.com/ads/library/?active_status=all&ad_type=political_and_issue_ads&country=FR
* https://www.facebook.com/ads/library/api/?source=archive-landing-page
    

In [1]:
import collections
import pprint

import requests
import pandas

import creds

In [2]:
FIELDS = [
    'ad_creation_time',
    'ad_creative_body',
    'ad_creative_link_caption',
    'ad_creative_link_description',
    'ad_creative_link_title',
    'ad_delivery_start_time',
    'ad_delivery_stop_time',
    'ad_snapshot_url',
    'currency',
    'demographic_distribution',
    'funding_entity',
    'impressions',
    'page_id',
    'page_name',
    'region_distribution',
    'spend',
]


In [25]:
def fetch():
    def make_request(after=None):
        params = {
            # 'ad-type': 'POLITICAL_AND_ISSUE_ADS' (default)
            'fields': ','.join(FIELDS),
            'search_terms': "''",
            'ad_reached_countries': "['GB']",
            'limit': 1000,
            'access_token': creds.FB_TOKEN,
        }
        if after:
            params['after'] = after

        response = requests.get(
            "https://graph.facebook.com/v3.3/ads_archive",
            params=params,
        )

        assert response.status_code == 200
        json_data = response.json()
        print('Got {} ads'.format(len(json_data['data'])))

        assert set(json_data) <= {'data', 'paging'}, set(json_data)

        ads = json_data['data']

        if 'paging' in json_data:
            paging = json_data['paging']
            assert set(paging) <= {'cursors', 'next', 'previous'}, paging
            assert set(paging['cursors']) <= {'after', 'before'}, paging
            after = json_data['paging']['cursors'].get('after')
        else:
            after = None
        

        return ads, after
    
    ads, after = make_request()
    while(after):
        ads_batch, after = make_request(after=after)
        ads += ads_batch

    return ads

In [26]:
ads = fetch()
len(ads)

Got 1000 ads
Got 993 ads
Got 907 ads
Got 0 ads


2900

In [27]:
df = pandas.DataFrame(ads)
df

,ad_creation_time,ad_creative_body,ad_creative_link_caption,ad_creative_link_description,ad_creative_link_title,ad_delivery_start_time,ad_delivery_stop_time,ad_snapshot_url,currency,demographic_distribution,funding_entity,impressions,page_id,page_name,region_distribution,spend
0,2019-05-15T08:45:34+0000,ISLAND’S MSP INVITES CABINET SECRETARY FOR RUR...,NaN,NaN,NaN,2019-05-15T08:45:36+0000,2019-05-18T08:45:34+0000,https://www.facebook.com/ads/archive/render_ad...,GBP,"[{'percentage': '0.333333', 'age': '65+', 'gen...",Kenneth Gibson,"{'lower_bound': '0', 'upper_bound': '999'}",2745800252158948,Isle of Arran SNP,"[{'percentage': '1', 'region': 'Scotland'}]","{'lower_bound': '0', 'upper_bound': '99'}"
1,2019-05-13T19:30:31+0000,Take the People's Vote test here in the North-...,NaN,NaN,NaN,2019-05-15T08:00:00+0000,2019-05-22T08:00:00+0000,https://www.facebook.com/ads/archive/render_ad...,GBP,"[{'percentage': '0.129032', 'age': '55-64', 'g...",Open Britain - West Cumbria,"{'lower_bound': '0', 'upper_bound': '999'}",267518870499295,Open Britain - West Cumbria,"[{'percentage': '0.724138', 'region': 'England...","{'lower_bound': '0', 'upper_bound': '99'}"
2,2019-05-15T02:30:22+0000,NaN,bit.ly,NaN,A collection of irreverent items to tell the w...,2019-05-15T07:01:37+0000,2019-05-18T07:00:00+0000,https://www.facebook.com/ads/archive/render_ad...,GBP,"[{'percentage': '0.175', 'age': '65+', 'gender...",Gutenberg Unbound (apolitical apparel and acce...,"{'lower_bound': '0', 'upper_bound': '999'}",2189982847933458,Gutenberg Unbound,"[{'percentage': '0.725', 'region': 'England'},...","{'lower_bound': '0', 'upper_bound': '99'}"
3,2019-05-15T06:35:36+0000,For radical Climate Action the UK must Remain ...,NaN,NaN,NaN,2019-05-15T06:35:39+0000,2019-05-24T06:35:36+0000,https://www.facebook.com/ads/archive/render_ad...,GBP,"[{'percentage': '0.117647', 'age': '35-44', 'g...",Karl Eslie Borges,"{'lower_bound': '0', 'upper_bound': '999'}",356980384912024,Alresford & Itchen Valley Greens,"[{'percentage': '1', 'region': 'England'}]","{'lower_bound': '0', 'upper_bound': '99'}"
4,2019-05-14T21:12:52+0000,https://www.change.org/p/give-victims-support-...,change.org,Episode 13 - Are HM Employment Judges Involved...,ARE HM EMPLOYMENT JUDGES INVOLVED IN CORRUPTIO...,2019-05-14T21:13:04+0000,2019-05-24T21:12:52+0000,https://www.facebook.com/ads/archive/render_ad...,GBP,"[{'percentage': '0.058824', 'age': '45-54', 'g...",Craig Chant,"{'lower_bound': '0', 'upper_bound': '999'}",195394467932052,Disabled Lives Matter,"[{'percentage': '1', 'region': 'England'}]","{'lower_bound': '0', 'upper_bound': '99'}"
5,2019-05-14T20:16:33+0000,***Footway Improvements in Llanrumney - Mount ...,NaN,NaN,NaN,2019-05-14T20:16:33+0000,2019-05-15T19:30:21+0000,https://www.facebook.com/ads/archive/render_ad...,GBP,"[{'percentage': '0.114848', 'age': '18-24', 'g...",Llanrumney Labour News,"{'lower_bound': '1000', 'upper_bound': '4999'}",1807970796086865,Llanrumney Labour News,"[{'percentage': '1', 'region': 'Wales'}]","{'lower_bound': '0', 'upper_bound': '99'}"
6,2019-05-14T19:59:49+0000,Creativity Movement; group which operates also...,NaN,NaN,NaN,2019-05-14T19:59:57+0000,2019-05-24T19:59:49+0000,https://www.facebook.com/ads/archive/render_ad...,GBP,"[{'percentage': '0.206897', 'age': '35-44', 'g...",Alberto Testa,"{'lower_bound': '0', 'upper_bound': '999'}",1899667790263172,Professor Alberto Testa,"[{'percentage': '0.714286', 'region': 'England...","{'lower_bound': '0', 'upper_bound': '99'}"
7,2019-05-14T19:33:15+0000,PINKSIXTY NEWS | 14 MAY 2019\n\n#LGBT rights d...,NaN,NaN,LGBT RIGHTS DECLINE ACROSS EUROPE,2019-05-14T19:33:25+0000,2019-05-15T19:33:15+0000,https://www.facebook.com/ads/archive/render_ad...,GBP,"[{'percentage': '0.060345', 'age': '65+', 'gen...",Leeze Lawrence,"{'lower_bound': '0', 'upper_bound': '999'}",108182029247820,Pinksixty,"[{'percentage': '0.836207', 'region': 'England...","{'lower_bound': '0', 'upper_bound': '99'}"
8,2019-05-14T19:10:42+0000,Cracking down on 

In [28]:
df[df['page_name']=='Pinksixty']

,ad_creation_time,ad_creative_body,ad_creative_link_caption,ad_creative_link_description,ad_creative_link_title,ad_delivery_start_time,ad_delivery_stop_time,ad_snapshot_url,currency,demographic_distribution,funding_entity,impressions,page_id,page_name,region_distribution,spend
7,2019-05-14T19:33:15+0000,PINKSIXTY NEWS | 14 MAY 2019\n\n#LGBT rights d...,NaN,NaN,LGBT RIGHTS DECLINE ACROSS EUROPE,2019-05-14T19:33:25+0000,2019-05-15T19:33:15+0000,https://www.facebook.com/ads/archive/render_ad...,GBP,"[{'percentage': '0.060345', 'age': '65+', 'gen...",Leeze Lawrence,"{'lower_bound': '0', 'upper_bound': '999'}",108182029247820,Pinksixty,"[{'percentage': '0.836207', 'region': 'England...","{'lower_bound': '0', 'upper_bound': '99'}"


In [100]:
def find_most_common(field):
    l = [
        ad[field]
        for ad in ads
        if field in ad
    ]
    c = collections.Counter(l)
    pprint.pprint(c.most_common(20))

In [101]:
find_most_common('funding_entity')

[('The Conservative Party', 1167),
 ('Conservatives', 345),
 ('Friends of the Earth', 315),
 ('the Liberal Democrats', 215),
 ('Change UK - The Independent Group', 205),
 ('QuoteSearch', 60),
 ('The Labour Party', 59),
 ('The Brexit Party', 54),
 ('Best for Britain', 39),
 ('Friends of the Earth England, Wales and Northern Ireland', 31),
 ('Terence Brotheridge', 26),
 ('Alexander Guy Dale', 13),
 ('crudelydrawn', 11),
 ("It's Our City", 10),
 ('First News ', 10),
 ('Terence Leonard Brotheridge', 9),
 ('Social Democratic and Labour Party (SDLP)', 8),
 ('The Independent Group (TIG) Ltd, company number 11770529, a registered '
  'company in England and Wales.',
  8),
 ('Uniunea Salvați România - USR', 7),
 ('The Conservative Party ', 7)]


In [103]:
find_most_common('page_name')

[('Conservatives', 447),
 ('Friends of the Earth', 346),
 ('Paul Bristow', 262),
 ('Change UK - The Independent Group', 213),
 ('Liberal Democrats', 209),
 ('Andy Street', 88),
 ('Robert Largan for High Peak', 68),
 ('Derek Thomas', 62),
 ('QuoteSearch.com', 60),
 ('Stuart Andrew', 60),
 ('Tom Hunt', 59),
 ('Peter Gibson For Darlington', 57),
 ('The Brexit Party', 54),
 ('Laura Wirral West', 54),
 ('Best For Britain', 39),
 ('The Labour Party', 37),
 ('Eddie Hughes MP', 16),
 ('Stuart Anderson', 16),
 ('Damien Moore MP', 15),
 ('Isobel Grant', 15)]
